In [2]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from PIL import Image
import numpy as np
import csv
import pandas as pd
from tensorflow.keras.utils import load_img
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [3]:
LETTERSTR = "0123456789abcdefghijklmnopqrstuvwxyz"


def toonehot(text):
    labellist = []
    for letter in text:
        onehot = np.array([0 for _ in range(36)])
        num = LETTERSTR.find(letter)
        onehot[num] = 1
        labellist.append(onehot)
    return np.array(labellist)


# Create CNN Model
print("Creating CNN model...")
in_ = Input((100, 100, 3))
out = in_
out = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Flatten()(out)
out = Dropout(0.3)(out)
out = Dense(36, name='digit1', activation='softmax')(out)
model = Model(inputs=in_, outputs=out)
model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['accuracy'])
# model.summary()


train_data = []
train_label = []
path = '/kaggle/input/captcha-hacker/train/task1'
df = pd.read_csv('/kaggle/input/annotation/annotations.csv')
for filename in os.listdir(path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img = load_img(os.path.join(path , filename) , target_size = (100 , 100))
        aug_images = []
        img = np.expand_dims(img , 0)
        gen = ImageDataGenerator(
        rotation_range = 10 , 
        width_shift_range = 0.15 , 
        height_shift_range = 0.15 , 
        shear_range = 0.15 , 
        zoom_range = 0.1 , 
        channel_shift_range = 10 , 
        )
        aug_iter = gen.flow(img)
        aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(5)]
        label = df.loc[df['filename'] == os.path.join('task1' , filename) , 'label'].item()
        label_ = toonehot(label)
        for aug_pic in np.array(aug_images):
            aug_pic = aug_pic.astype('float')/255.0
            train_data.append(aug_pic)
            train_label.append(label_[0])
train_data = np.array(train_data)
train_label = np.array(train_label)
# print(train_data.shape)
# print(train_label.shape)
# train_data = tf.stack(train_data)
# train_label = tf.stack(train_label)
# print (train_data.shape)

model.fit(train_data, train_label, batch_size = 100, epochs = 16 , verbose=1 , validation_split=0.2) 
model.save("part1_model.h5")         

Creating CNN model...


2022-12-13 10:49:23.488925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 10:49:23.489854: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 10:49:23.776373: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 10:49:23.777644: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 10:49:23.778809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Epoch 1/16


2022-12-13 10:50:17.383500: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


80/80 [==============================] - 23s 92ms/step - loss: 3.0684 - accuracy: 0.1971 - val_loss: 5.6954 - val_accuracy: 0.0000e+00
Epoch 2/16
80/80 [==============================] - 7s 87ms/step - loss: 1.9378 - accuracy: 0.3885 - val_loss: 5.6635 - val_accuracy: 0.0000e+00
Epoch 3/16
80/80 [==============================] - 7s 85ms/step - loss: 1.2545 - accuracy: 0.5784 - val_loss: 4.7232 - val_accuracy: 0.1120
Epoch 4/16
80/80 [==============================] - 7s 85ms/step - loss: 0.8880 - accuracy: 0.7005 - val_loss: 3.8643 - val_accuracy: 0.2380
Epoch 5/16
80/80 [==============================] - 7s 85ms/step - loss: 0.6371 - accuracy: 0.7855 - val_loss: 2.3156 - val_accuracy: 0.3435
Epoch 6/16
80/80 [==============================] - 7s 88ms/step - loss: 0.4698 - accuracy: 0.8445 - val_loss: 1.7651 - val_accuracy: 0.4620
Epoch 7/16
80/80 [==============================] - 7s 85ms/step - loss: 0.3629 - accuracy: 0.8773 - val_loss: 0.7473 - val_accuracy: 0.7545
Epoch 8/16
80/8